In [88]:
import os
import numpy as np 
import pandas as pd 
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
warnings.filterwarnings("ignore")
pd.set_option("display.max_rows",None)
from sklearn import preprocessing
import matplotlib 
matplotlib.style.use('ggplot')
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
import chardet


In [19]:
# Detectar la codificación del archivo
with open("DatosEjemplo.csv", 'rb') as f:
    result = chardet.detect(f.read())

In [20]:
# Utilizar la codificación detectada para leer el archivo
try:
    df = pd.read_csv("DatosEjemplo.csv", encoding=result['encoding'],  sep=';', decimal=',')
    print("Archivo cargado exitosamente.")
except Exception as e:
    print(f"Error al cargar el archivo: {e}")

Archivo cargado exitosamente.


In [21]:
# Convertir las columnas de tipo object a string
#string_col = df.select_dtypes(include="object").columns
#df[string_col]=df[string_col].astype("string")

In [22]:
df = df.drop(['Paciente'], axis=1)
string_col=df.select_dtypes("object").columns.to_list()

In [23]:
df[string_col].head()
for col in string_col:
    print(f"The distribution of categorical valeus in the {col} is : ")
    print(df[col].value_counts())

The distribution of categorical valeus in the Género is : 
Género
F    118
M     45
Name: count, dtype: int64


In [24]:
#Aqui intentamos pasar el genero de cualitativo a cuantitativo
# Applying the condition
df.loc[df["Género"] == "M", "Género"] = 1
df["Género"] = np.where(df["Género"] == "F", 0, 1)
df.head()

,Edad (años),Género,Altura (cm),Peso (Kg),Indice de Masa Corporal,Velocidad marcha (m/s),Fragilidad
0,82,0,1.62,59.0,22.48,1.42,0
1,82,1,1.70,67.0,23.18,0.93,0
2,77,0,1.71,68.0,23.26,0.83,0
3,82,1,1.60,52.0,20.31,0.74,0
4,97,1,1.70,93.0,32.18,0.64,0


In [66]:
x = df.drop("Fragilidad", axis =1) #Variables independientes
y = df["Fragilidad"] #Varaibkle dependiente

In [67]:
#Normalizamos los datos
norm = StandardScaler()
x_normalizado = norm.fit_transform(x)

In [68]:
#Divido los datos en conjuntos de entrenamiento y prueba
x_train, x_test, y_train, y_test = train_test_split(x_normalizado,y,test_size=0.3,random_state=1)

#Creo el modelo de KNN
knn = KNeighborsClassifier(metric = 'euclidean')
#Ajusto el modelo a los datos de entrenamiento
knn.fit(x_train, y_train)

# Hago predicciones sobre los datos de prueba
y_pred_knn = knn.predict(x_test)

In [69]:
y_pred_knn

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0], dtype=int64)

In [80]:
#Matriz de confusion, cuyo resultado nos da que tenemos varios falsos positivos y algunos verdaderos negativos pero ningun falso negativo (buenas noticias)
print(confusion_matrix(y_test,y_pred_knn))

[[35  0]
 [12  2]]


In [76]:
#Caluculamos el acurracy que da lo precision de verdaderos positivos y verdaderos negativos
print(accuracy_score(y_test,y_pred_knn))
#Caluclamos el precision que da el numero de verdaderos positivos respecto al total de datos teniendo en cuenta solo los verdaderos positivos
print(precision_score(y_test,y_pred_knn))
#Calcula la sensibilidad, es decir, que tan bueno es el modelo para clasificar correctamente un resultado considerando tambien los falsos negativos
print(recall_score(y_test,y_pred_knn))

0.7551020408163265
1.0
0.14285714285714285


In [84]:
#Ahora con el Modelo Bernoulli Naive Bayes

In [85]:
# Calculo la mediana de los datos de entrenamiento
mediana = np.median(x_train)

# Binarizando los datos de entrenamiento usando la mediana
X_train_binarizado = np.where(x_train > mediana, 1, 0)

# Binarizando los datos de prueba usando la mediana de los datos de entrenamiento
X_test_binarizado = np.where(x_test > mediana, 1, 0)

# Creo el modelo
bnb = BernoulliNB()

# Entreno el modelo
bnb.fit(X_train_binarizado, y_train)

# Predicción
prediccion_BNb = bnb.predict(X_test_binarizado)

In [87]:
#Matriz de confusion, cuyo resultado nos da que tenemos varios falsos positivos y algunos verdaderos negativos pero ningun falso negativo (buenas noticias)
print(confusion_matrix(y_test,prediccion_BNb))

[[35  0]
 [14  0]]


In [86]:
#Caluculamos el acurracy que da lo precision de verdaderos positivos y verdaderos negativos
print(accuracy_score(y_test,prediccion_BNb))
#Caluclamos el precision que da el numero de verdaderos positivos respecto al total de datos teniendo en cuenta solo los verdaderos positivos
print(precision_score(y_test,prediccion_BNb))
#Calcula la sensibilidad, es decir, que tan bueno es el modelo para clasificar correctamente un resultado considerando tambien los falsos negativos
print(recall_score(y_test,prediccion_BNb))

0.7142857142857143
0.0
0.0


In [89]:
#Metodo Simbolico

In [90]:
#iniciando el modelo
dtc = DecisionTreeClassifier(criterion='entropy', random_state=42)
#entrenando el modelo
dtc.fit(x_train, y_train)
#verificando la importancia de cada atributo
dtc.feature_importances_
prediccion_ArbolDecision = dtc.predict(x_test)

In [92]:
#Matriz de confusion, cuyo resultado nos da que tenemos varios falsos positivos y algunos verdaderos negativos pero ningun falso negativo (buenas noticias)
print(confusion_matrix(y_test,prediccion_ArbolDecision))

[[27  8]
 [11  3]]


In [91]:
#Caluculamos el acurracy que da lo precision de verdaderos positivos y verdaderos negativos
print(accuracy_score(y_test,prediccion_ArbolDecision))
#Caluclamos el precision que da el numero de verdaderos positivos respecto al total de datos teniendo en cuenta solo los verdaderos positivos
print(precision_score(y_test,prediccion_ArbolDecision))
#Calcula la sensibilidad, es decir, que tan bueno es el modelo para clasificar correctamente un resultado considerando tambien los falsos negativos
print(recall_score(y_test,prediccion_ArbolDecision))

0.6122448979591837
0.2727272727272727
0.21428571428571427


In [ ]:
#Evidentemente existen numerosos hiperparametros que se podrian modificar para obtener mejores preciones
#en nuestro modelo, sin embargo, en este caso realizaremos simplemente un analisis superficial para mostrar
#el potencial del mismo.